In [ ]:
df1 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="1 bedroom flat")
df2 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="2 bedroom flat")
df3 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="3 bedroom flat")
df4 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="2 bedroom house")
df5 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="3 bedroom house")
df6 = pd.read_excel("../../data/raw/moving-annual-rent-2025.xlsx", sheet_name="4 bedroom house")

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
import pmdarima as pm
import matplotlib.pyplot as plt

In [18]:
import pandas as pd
import os

file_path = "../../data/raw/moving-annual-rent-2025.xlsx"
output_dir = "../../data/processed/forecast"

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

xls = pd.ExcelFile(file_path)

# Get all sheets except "All properties"
sheets = [s for s in xls.sheet_names if s != "All properties"]
print("Sheets included:", sheets)

for sheet in sheets:
    df_raw = pd.read_excel(file_path, sheet_name=sheet, header=None)

    # Suburbs (col 1, from row 3 onward)
    suburbs = df_raw.iloc[3:, 1].reset_index(drop=True)
    suburbs.name = "Suburb"

    # Date row and type row
    date_row = df_raw.iloc[1]
    type_row = df_raw.iloc[2]

    # Keep only Median columns
    median_cols = [i for i, t in enumerate(type_row) if t == "Median"]

    data = {}
    data["Suburb"] = suburbs

    for col in median_cols:
        date_str = str(date_row[col]).strip()
        try:
            date = pd.to_datetime(date_str, format="%b %Y")
            date_label = date.strftime("%Y-%m")  # e.g. "2000-03"
        except:
            continue

        data[date_label] = pd.to_numeric(df_raw.iloc[3:, col].reset_index(drop=True), errors="coerce")

    df_wide = pd.DataFrame(data)

    # Drop "Group Total" if present
    df_wide = df_wide[df_wide["Suburb"] != "Group Total"]

    # Save to ../../processed/forecast
    out_file = os.path.join(output_dir, f"{sheet.replace(' ', '_')}.csv")
    df_wide.to_csv(out_file, index=False)
    print(f"Saved {out_file} with shape {df_wide.shape}")

Sheets included: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house']
Saved ../../data/processed/forecast/1_bedroom_flat.csv with shape (146, 102)
Saved ../../data/processed/forecast/2_bedroom_flat.csv with shape (146, 102)
Saved ../../data/processed/forecast/3_bedroom_flat.csv with shape (146, 102)
Saved ../../data/processed/forecast/2_bedroom_house.csv with shape (146, 102)
Saved ../../data/processed/forecast/3_bedroom_house.csv with shape (146, 102)
Saved ../../data/processed/forecast/4_bedroom_house.csv with shape (146, 102)
